# TEST MODE

In [1]:
# 30 min + 30 + 10 + 

In [2]:
TEST_MODE=True

# Сиды

In [3]:
import numpy as np
import random
import os

seed=105

os.environ['PYTHONHASHSEED']=str(seed)
random.seed(seed)
np.random.seed(seed)


# Импорты

In [4]:
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.model_selection import StratifiedKFold, train_test_split, KFold
from catboost import CatBoostClassifier, Pool, CatBoostRegressor
from transformers import AutoModelForSequenceClassification, AutoTokenizer


# Загрузка данных

In [5]:
train_stat=pd.read_parquet('/kaggle/input/attention-on-screen/video_stat (1).parquet')
train=pd.read_parquet('/kaggle/input/attention-on-screen/train (6).parquet')
test=pd.read_parquet('/kaggle/input/attention-on-screen/test (1).parquet')
sample=pd.read_csv('/kaggle/input/attention-on-screen/sample_submission (14).csv')


In [6]:
num_feats=train.select_dtypes(include='number').columns.tolist()

cat_feats=train.select_dtypes(include='object').columns.tolist()

In [7]:
num2_feats=train_stat.select_dtypes(include='number').columns.tolist()

In [8]:
if TEST_MODE:
    train=train[:3_000_000]
    

# EDA

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 6 columns):
 #   Column           Dtype                        
---  ------           -----                        
 0   event_timestamp  datetime64[ms, Europe/Moscow]
 1   user_id          object                       
 2   region           object                       
 3   city             object                       
 4   video_id         object                       
 5   watchtime        int64                        
dtypes: datetime64[ms, Europe/Moscow](1), int64(1), object(4)
memory usage: 137.3+ MB


# Joining stuff

In [10]:
#train_full=train.merge(train_stat, on=['video_id'], how='left')

test_full=test.merge(train_stat, on=['video_id'], how='left')

In [11]:
#train_full.to_csv('for_analysis.csv', index=False)

# Still EDA

In [12]:
#v_duration                               0.004272
#v_likes                                  0.001689

In [13]:
import pandas as pd
import numpy as np

# 1) Keep only train rows whose user is present in test
#users_in_test = pd.Index(test['user_id'].dropna().unique())
#train_new = train[train['user_id'].isin(users_in_test)].copy()

#print(f"train rows:     {len(train):,}")
#print(f"train_new rows: {len(train_new):,} ({len(train_new)/len(train):.2%} of train)\n")

# 2) Helper to report both unique-level and row-level coverage
def coverage_stats(train_series: pd.Series, test_series: pd.Series, name: str):
    tr_u = pd.Index(train_series.dropna().unique())
    te_u = pd.Index(test_series.dropna().unique())
    n_common_u = tr_u.intersection(te_u).size
    pct_common_u = n_common_u / te_u.size if te_u.size else np.nan

    # Row-level: how many test rows have a value that exists in train_new
    covered_rows = test_series.isin(tr_u).sum()
    pct_rows = covered_rows / len(test_series)
    unseen_rows = len(test_series) - covered_rows

    print(f"{name}:")
    print(f"  test unique:          {te_u.size:,}")
    print(f"  common unique:        {n_common_u:,}  ({pct_common_u:.2%})")
    print(f"  test rows covered:    {covered_rows:,}  ({pct_rows:.2%})")
    print(f"  unseen test rows:     {unseen_rows:,}  ({1 - pct_rows:.2%})\n")

# 3) Re-check coverage after filtering train -> train_new
#coverage_stats(train_new['event_timestamp'], test['event_timestamp'], "event_timestamp")
#coverage_stats(train_new['video_id'],         test['video_id'],         "video_id")


# Feature engineering

In [14]:
train_full_new=train.merge(train_stat, on=['video_id'], how='left')

In [15]:
useless=['v_cr_click_dislike_7_days','v_cr_click_dislike_30_days',
         'v_avg_watchtime_7_day','v_avg_watchtime_30_day',
         'v_frac_avg_watchtime_7_day_duration','v_frac_avg_watchtime_30_day_duration',
         'v_month_views','v_week_views',
         'v_cr_click_like_7_days','v_cr_click_like_30_days',
         'v_cr_click_comment_7_days','v_cr_click_comment_30_days',
         'v_cr_click_long_view_7_days','v_cr_click_long_view_30_days',
         'v_category_popularity_percent_30_days',
         'v_long_views_7_days','v_long_views_30_days',
         'v_is_deleted','v_is_hidden','row_number'
         
        # 'user_id','category_id','author_id', 'video_id',
         #'description','title',
        # 'region','city'
        ]

In [16]:
train_full_new1=train_full_new.drop(columns=useless)

test_full_new1=test_full.drop(columns=useless)

In [17]:
import numpy as np
import pandas as pd

import numpy as np
import pandas as pd

def make_long_watch_flag(df, duration_col='v_duration', watch_col='watchtime', strict=True, unit='s'):
    # 1) to seconds (if needed) and numeric
    dur = pd.to_numeric(df[duration_col], errors='coerce')
    wt  = pd.to_numeric(df[watch_col],  errors='coerce')
    if unit == 'ms':
        dur = dur / 1000.0
        wt  = wt  / 1000.0

    # 2) thresholds
    thr = np.where(dur > 300.0, dur * 0.25, 30.0)  # 300s = 5 min

    # 3) validity mask and comparison
    valid = dur.notna() & wt.notna() & (dur > 0) & (wt >= 0)
    cmp = (wt > thr) if strict else (wt >= thr)

    # 4) build nullable Int8 target safely (pandas Series + pd.NA)
    y = pd.Series(cmp, index=df.index)
    y.loc[~valid] = pd.NA
    df['y_long'] = y.astype('Int8')  # 0/1 with NA

    # optional: keep threshold for debugging
    df['longwatch_threshold_sec'] = pd.Series(np.where(valid, thr, np.nan), index=df.index)

    return df


# Example:
train_full_new1 = make_long_watch_flag(train_full_new1, 'v_duration', 'watchtime', strict=True)
# If test has watchtime and you want the same flag there:
# test = make_long_watch_flag(test, 'v_duration', 'watchtime', strict=True)

# Quick sanity check
print(train_full_new1['y_long'].value_counts(dropna=False))


/tmp/ipykernel_37/1262398379.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  y.loc[~valid] = pd.NA


y_long
1       1474075
0       1430935
<NA>      94990
Name: count, dtype: Int64


In [18]:
# Вариант 1 (рекомендую)
before = len(train_full_new1)
train_full_new1 = train_full_new1.dropna(subset=['y_long']).copy()
train_full_new1['y_long'] = train_full_new1['y_long'].astype('int8')
print(f"Dropped {before - len(train_full_new1)} rows")


Dropped 94990 rows


In [19]:
'''
Взять title и получить эмбеды 
Всякие лаг фичи, фурье фичи, по таймстемпу, цикличные фичи а также бинарные фичи по типу всякие праздники по типу нг 
по id создателей сделать агреггированные фичи 

Для видео продолжительностью более 5 минут: просмотр считается долгим, если пользователь посмотрел больше 25% длительности видео.
Для видео менее или равные 5 минут: просмотр считается долгим, если пользователь посмотрел более 30 секунд.

Разница между event_timestamp и v_pub_timestamp

Надо попробовать что Даня сказал с SSL и вот сделать тут также с текстом!
'''

'\nВзять title и получить эмбеды \nВсякие лаг фичи, фурье фичи, по таймстемпу, цикличные фичи а также бинарные фичи по типу всякие праздники по типу нг \nпо id создателей сделать агреггированные фичи \n\nДля видео продолжительностью более 5 минут: просмотр считается долгим, если пользователь посмотрел больше 25% длительности видео.\nДля видео менее или равные 5 минут: просмотр считается долгим, если пользователь посмотрел более 30 секунд.\n\nРазница между event_timestamp и v_pub_timestamp\n\nНадо попробовать что Даня сказал с SSL и вот сделать тут также с текстом!\n'

In [20]:
'''
Хорошие фичи

v_duration                               0.004272
v_likes                                  0.001689
'''

'\nХорошие фичи\n\nv_duration                               0.004272\nv_likes                                  0.001689\n'

In [21]:
'''
Тест и трейн один и тот же день. Надо об этом учитывать и делать фичи именно сильно дающие инфу о 2024-08-10 
А также получается надо минутные цикличные или часовые(Надо построить спец график(есть имя ему) )
'''

'\nТест и трейн один и тот же день. Надо об этом учитывать и делать фичи именно сильно дающие инфу о 2024-08-10 \nА также получается надо минутные цикличные или часовые(Надо построить спец график(есть имя ему) )\n'

In [22]:
#Регрессия или классификация? Склоняюсь к регрессии и потом просто переводе в нули и единички 

In [23]:
import re
import numpy as np
import pandas as pd
from typing import Tuple, Dict, Any, Optional

# --------- helpers & regex ----------
_RX_EMOJI = re.compile(
    "[" "\U0001F300-\U0001F64F" "\U0001F680-\U0001F6FF" "\U0001F900-\U0001F9FF"
    "\u2600-\u26FF\u2700-\u27BF" "]+"
)

# curated keyword hints (RU+EN)
_HINT_SHORT  = r"(?:\bshorts?\b|шортс|тизер|teaser|трейлер|trailer|highlights?|нарезка|clip\b|клип\b)"
_HINT_LONG   = r"(?:стрим|прямой эфир|запись стрима|live|stream|подкаст|podcast|full(?:\s+version|\s+episode)?|полная верс|полный выпуск|longplay)"
_HINT_EDU    = r"(?:\bкак\b|how to|tutorial|урок|гайд|инструкц|обзор|review)"
_HINT_OFFCL  = r"(?:official|официальн)"
_RX_PART_EP  = re.compile(r"(?:\bpart\s*\d+\b|\bчаст[ьи]\s*\d+\b|\bs\d{1,2}e\d{1,2}\b)", re.I)
_RX_YEAR     = re.compile(r"\b(19\d{2}|20\d{2}|210\d)\b")
_RX_DATE_DDMM = re.compile(r"\b([0-3]?\d)[\./\-]([01]?\d)\b")  # 12.08 or 12-8

# time mentions in title
_RX_HHMMSS = re.compile(r"\b(\d{1,2}):([0-5]\d)(?::([0-5]\d))?\b")
_RX_HOURS  = re.compile(r"(\d+)\s*(?:час|часа|часов|h|hr|hour)s?\b", re.I)
_RX_MIN    = re.compile(r"(\d+)\s*(?:мин|минут|minute|m)\b", re.I)
_RX_SEC    = re.compile(r"(\d+)\s*(?:сек|секунд|second|s)\b", re.I)

def _normalize_title(s: pd.Series) -> pd.Series:
    s = s.fillna("").astype(str)
    s = s.str.replace(r"\s+", " ", regex=True).str.strip()
    return s

def _extract_title_time_sec(title: str) -> float:
    """Parse a rough duration from title; return seconds or np.nan."""
    if not title:
        return np.nan
    t = title.lower()

    # HH:MM(:SS)
    m = _RX_HHMMSS.search(t)
    if m:
        h = int(m.group(1))
        mm = int(m.group(2))
        ss = int(m.group(3)) if m.group(3) else 0
        return h*3600 + mm*60 + ss

    total_sec = 0.0
    found = False
    for g in _RX_HOURS.findall(t):
        total_sec += float(g) * 3600; found = True
    for g in _RX_MIN.findall(t):
        total_sec += float(g) * 60;   found = True
    for g in _RX_SEC.findall(t):
        total_sec += float(g);        found = True
    return total_sec if found else np.nan

# --------- main builders ----------
def _fast_title_core(df: pd.DataFrame, title_col: str = "title") -> pd.DataFrame:
    out = df.copy()
    t_orig = out[title_col].fillna("").astype(str)
    t = _normalize_title(out[title_col]).str.lower()

    # lengths
    out["title_len_char"] = t.str.len().astype("int32")
    words = t.str.split()
    out["title_n_words"]  = words.str.len().fillna(0).astype("int16")
    out["title_mean_word_len"] = (
        np.where(out["title_n_words"]>0,
                 out["title_len_char"]/out["title_n_words"].clip(lower=1),
                 0).astype("float32")
    )

    # counts/ratios (use original to preserve case/punct)
    denom = out["title_len_char"].replace(0, np.nan)
    out["title_digits_ratio"] = t_orig.str.count(r"\d").div(denom).fillna(0).astype("float32")
    out["title_punct_ratio"]  = t_orig.str.count(r"[^\w\s]", flags=re.UNICODE).div(denom).fillna(0).astype("float32")
    out["title_excl_cnt"]     = t_orig.str.count("!").astype("int16")
    out["title_quest_cnt"]    = t_orig.str.count(r"\?").astype("int16")
    out["title_hash_cnt"]     = t_orig.str.count("#").astype("int16")
    out["title_at_cnt"]       = t_orig.str.count("@").astype("int16")
    out["title_emoji_cnt"]    = t_orig.apply(lambda x: len(_RX_EMOJI.findall(x))).astype("int16")

    # scripts / language hints
    has_cyr = t_orig.str.contains(r"[А-Яа-яЁё]")
    has_lat = t_orig.str.contains(r"[A-Za-z]")
    out["title_has_cyr"]  = has_cyr.astype("int8")
    out["title_has_lat"]  = has_lat.astype("int8")
    out["title_has_mixed"] = (has_cyr & has_lat).astype("int8")

    # curated keyword flags (compiled patterns are already fast)
    out["title_is_shorts"]   = t.str.contains(_HINT_SHORT,  regex=True).astype("int8")
    out["title_is_longform"] = t.str.contains(_HINT_LONG,   regex=True).astype("int8")
    out["title_is_edu"]      = t.str.contains(_HINT_EDU,    regex=True).astype("int8")
    out["title_is_official"] = t.str_contains(_HINT_OFFCL,  regex=True).astype("int8") if hasattr(t, 'str_contains') else t.str.contains(_HINT_OFFCL, regex=True).astype("int8")

    out["title_has_part_ep"] = t.str.contains(_RX_PART_EP).astype("int8")
    out["title_has_year"]    = t.str.contains(_RX_YEAR).astype("int8")
    out["title_has_date"]    = t.str.contains(_RX_DATE_DDMM).astype("int8")

    # time mention from title
    out["title_time_sec"]    = t.apply(_extract_title_time_sec).astype("float32")
    out["title_has_time"]    = out["title_time_sec"].notna().astype("int8")

    # starts-with-number / all-caps token presence
    out["title_starts_num"]  = t.str.match(r"^\s*\d").fillna(False).astype("int8")
    out["title_has_topN"]    = t.str.contains(r"\btop[- ]?\d+\b|\bтоп[- ]?\d+\b", regex=True).astype("int8")

    # binned lengths (cheap & strong for trees)
    out["title_len_char_bin"] = pd.cut(out["title_len_char"], [-1,20,35,60,100,10_000], labels=[0,1,2,3,4]).astype("int8")
    out["title_n_words_bin"]  = pd.cut(out["title_n_words"],  [-1,3,6,12,20,10_000],   labels=[0,1,2,3,4]).astype("int8")

    return out

def _title_freq_from_train(train_norm_titles: pd.Series) -> pd.Series:
    vc = train_norm_titles.value_counts()
    return vc

def add_fast_title_features(
    train_df: pd.DataFrame,
    test_df: pd.DataFrame,
    title_col: str = "title",
    v_duration_col: Optional[str] = "v_duration",  # seconds; set None if absent
    add_tfidf_svd: bool = False,                   # optional tiny SVD
    tfidf_max_features: int = 15000,
    svd_components: int = 16,
) -> Tuple[pd.DataFrame, pd.DataFrame, Dict[str, Any]]:
    """
    Fast, high-signal title features with zero heavy deps; optional tiny TF-IDF+SVD.
    Returns (train_out, test_out, artifacts).
    """
    tr = _fast_title_core(train_df, title_col=title_col)
    te = _fast_title_core(test_df,  title_col=title_col)

    # title frequency (from train) -> map to both
    tnorm_tr = _normalize_title(tr[title_col]).str.lower()
    tnorm_te = _normalize_title(te[title_col]).str.lower()
    freq = _title_freq_from_train(tnorm_tr)
    tr["title_freq_train"] = tnorm_tr.map(freq).fillna(0).astype("int32")
    te["title_freq_train"] = tnorm_te.map(freq).fillna(0).astype("int32")
    tr["title_freq_log1p"] = np.log1p(tr["title_freq_train"]).astype("float32")
    te["title_freq_log1p"] = np.log1p(te["title_freq_train"]).astype("float32")
    tr["title_is_duplicated"] = (tr["title_freq_train"] > 1).astype("int8")
    te["title_is_duplicated"] = (te["title_freq_train"] > 1).astype("int8")

    # title vs meta duration (if you have v_duration in seconds)
    if v_duration_col and (v_duration_col in tr.columns):
        vd_tr = pd.to_numeric(tr[v_duration_col], errors="coerce").astype("float32")
        vd_te = pd.to_numeric(te.get(v_duration_col, np.nan), errors="coerce").astype("float32") if v_duration_col in te.columns else pd.Series(np.nan, index=te.index, dtype="float32")

        # only if both present
        for df, vd in ((tr, vd_tr), (te, vd_te)):
            has_both = df["title_time_sec"].notna() & vd.notna() & (vd > 0)
            ratio = pd.Series(np.nan, index=df.index, dtype="float32")
            gap   = pd.Series(np.nan, index=df.index, dtype="float32")
            ratio[has_both] = (df.loc[has_both, "title_time_sec"] / vd.loc[has_both]).astype("float32")
            gap[has_both]   = (df.loc[has_both, "title_time_sec"] - vd.loc[has_both]).astype("float32")
            df["title_time_ratio_to_meta"] = ratio
            df["title_time_gap_to_meta"]   = gap
            # safe caps
            df["title_time_ratio_to_meta_c"] = df["title_time_ratio_to_meta"].clip(0, 5).astype("float32")
            df["title_time_gap_to_meta_c"]   = df["title_time_gap_to_meta"].clip(-3*3600, 3*3600).astype("float32")
            df["title_time_consistent"]      = has_both.astype("int8")

    artifacts: Dict[str, Any] = {"title_freq": freq}

    # optional: tiny TF-IDF + SVD (still pretty fast, big boost)
    if add_tfidf_svd:
        from sklearn.feature_extraction.text import TfidfVectorizer
        from sklearn.decomposition import TruncatedSVD

        vec = TfidfVectorizer(
            analyzer="char",
            ngram_range=(3,5),
            max_features=tfidf_max_features,
            min_df=2
        )
        X_tr = vec.fit_transform(tnorm_tr)
        X_te = vec.transform(tnorm_te)

        svd = TruncatedSVD(n_components=svd_components, random_state=42)
        Z_tr = svd.fit_transform(X_tr).astype("float32")
        Z_te = svd.transform(X_te).astype("float32")

        for i in range(svd_components):
            tr[f"title_svd_{i:02d}"] = Z_tr[:, i]
            te[f"title_svd_{i:02d}"] = Z_te[:, i]

        artifacts.update({"tfidf_vectorizer": vec, "svd": svd})

    return tr, te, artifacts


In [24]:
# Minimal fast set (no heavy libs)
'''
train_title_fast, test_title_fast, title_arts = add_fast_title_features(
    train_full_new1, test_full_new1,
    title_col="title",
    v_duration_col="v_duration",   # or None if not available
    add_tfidf_svd=True            # set True for a small extra boost
)

# Peek
print(train_title_fast.filter(regex="^title_").columns.tolist()[:30])
print(train_title_fast.filter(regex="^title_").describe().T.head())
'''

'\ntrain_title_fast, test_title_fast, title_arts = add_fast_title_features(\n    train_full_new1, test_full_new1,\n    title_col="title",\n    v_duration_col="v_duration",   # or None if not available\n    add_tfidf_svd=True            # set True for a small extra boost\n)\n\n# Peek\nprint(train_title_fast.filter(regex="^title_").columns.tolist()[:30])\nprint(train_title_fast.filter(regex="^title_").describe().T.head())\n'

In [25]:
import math
from collections import defaultdict, deque
from dataclasses import dataclass, field
from typing import Dict, Optional, Iterable, Tuple

import numpy as np
import pandas as pd


# ============================ utils ============================

EPS = 1e-6

def safe_div(a, b, default=np.nan):
    return a / b if (b is not None and b != 0) else default

def logit(p: float) -> float:
    p = min(max(p, EPS), 1 - EPS)
    return math.log(p / (1 - p))

def invlogit(x: float) -> float:
    return 1 / (1 + math.exp(-x))

def pct01(x: pd.Series) -> pd.Series:
    """Привести проценты к [0,1] если они заданы в [0,100]."""
    if x.max(skipna=True) is None:
        return x
    mx = x.max(skipna=True)
    return x/100.0 if (mx is not None and mx > 1.0) else x

def beta_smooth(success, total, alpha, p0):
    return (success + alpha * p0) / (total + alpha + EPS)

def to_dt(s):
    """Поддержка int timestamp (sec) или уже datetime."""
    if np.issubdtype(s.dtype, np.integer) or np.issubdtype(s.dtype, np.floating):
        return pd.to_datetime(s, unit="s", utc=True)
    elif np.issubdtype(s.dtype, np.datetime64):
        return pd.to_datetime(s, utc=True)
    else:
        # строка
        return pd.to_datetime(s, utc=True)

def cos_sim(a: np.ndarray, b: np.ndarray) -> float:
    if a is None or b is None:
        return np.nan
    na = np.linalg.norm(a)
    nb = np.linalg.norm(b)
    if na < 1e-12 or nb < 1e-12:
        return np.nan
    return float(np.dot(a, b) / (na * nb))

def ema_update(vec_old: Optional[np.ndarray], vec_new: Optional[np.ndarray], beta: float) -> Optional[np.ndarray]:
    if vec_new is None:
        return vec_old
    if vec_old is None:
        return vec_new.copy()
    return (1 - beta) * vec_old + beta * vec_new

def ensure_array(x):
    if x is None:
        return None
    if isinstance(x, (list, tuple)):
        return np.asarray(x, dtype="float32")
    if isinstance(x, np.ndarray):
        return x.astype("float32")
    # если в колонке сериализованная строка — попробуем ast.literal_eval снаружи
    return np.asarray(x, dtype="float32")


# ============================ config ============================

@dataclass
class FeatConfig:
    session_gap_s: int = 30 * 60  # 30 минут
    user_L_list: Tuple[int, ...] = (1, 3, 5, 10, 30)
    alpha_video: float = 50.0
    alpha_author: float = 200.0
    alpha_category: float = 500.0
    alpha_online_small: float = 3.0     # сглаживание для user×pair/geo и т.п.
    ema_beta: float = 0.2               # EMA для пользовательских центроидов
    min_views_video: int = 50           # пороги backoff
    min_views_author: int = 200
    min_views_category: int = 1000
    fresh_hours: int = 24               # что считаем "свежим" видео
    recent_k_sim: int = 5               # k в "user_recent_k_sim"
    make_time_bins: bool = True


# ============================ main ============================

def featurize(
    df_train: pd.DataFrame,
    df_test: pd.DataFrame,
    video_stat: pd.DataFrame,
    video_emb: Optional[pd.DataFrame] = None,   # опционально: columns ['video_id','emb'] где emb—np.ndarray
    cfg: FeatConfig = FeatConfig(),
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Возвращает (train_feat, test_feat) cо всеми перечисленными golden-фичами.
    Ожидаемые колонки:
      - df_train: ['event_timestamp','video_id','user_id','region','city','watchtime']
      - df_test : ['event_timestamp','video_id','user_id','region','city']
      - video_stat: перечисленные в условии (duration, views, long_views, cr, dates, author_id, category_id, ...)

    Если передан video_emb: должен содержать по крайней мере ['video_id','emb'].
    """

    # ---------- 0) Копии и базовые преобразования ----------
    train = df_train.copy()
    test  = df_test.copy()
    vs    = video_stat.copy()

    # timestamps to UTC datetime
    train["event_dt"] = to_dt(train["event_timestamp"])
    test["event_dt"]  = to_dt(test["event_timestamp"])
    vs["v_pub_datetime"] = to_dt(vs["v_pub_datetime"])

    # duration
    vs["v_duration"] = vs["v_duration"].astype(float)

    # просмотры по окнам
    vs["views_1d"]  = vs["v_day_views"].fillna(0).astype(float)
    vs["views_7d"]  = vs["v_week_views"].fillna(0).astype(float)
    vs["views_30d"] = vs["v_month_views"].fillna(0).astype(float)

    # long views по окнам
    vs["lv_1d"]  = vs["v_long_views_1_days"].fillna(0).astype(float)
    vs["lv_7d"]  = vs["v_long_views_7_days"].fillna(0).astype(float)
    vs["lv_30d"] = vs["v_long_views_30_days"].fillna(0).astype(float)

    # глобальный p0 по 30d (если нет — fallback 0.2)
    sum_lv30  = vs["lv_30d"].sum()
    sum_v30   = vs["views_30d"].sum()
    p0_global = safe_div(sum_lv30, sum_v30, default=0.2)
    p0_global = float(0.2 if (p0_global is np.nan or p0_global is None) else p0_global)

    # ---------- 1) p_vid_* c бета-сглаживанием + тренды ----------
    vs["p_vid_1d"]  = beta_smooth(vs["lv_1d"],  vs["views_1d"],  cfg.alpha_video, p0_global)
    vs["p_vid_7d"]  = beta_smooth(vs["lv_7d"],  vs["views_7d"],  cfg.alpha_video, p0_global)
    vs["p_vid_30d"] = beta_smooth(vs["lv_30d"], vs["views_30d"], cfg.alpha_video, p0_global)

    vs["logit_p_vid_1d"]  = vs["p_vid_1d"].clip(EPS, 1-EPS).map(logit)
    vs["logit_p_vid_7d"]  = vs["p_vid_7d"].clip(EPS, 1-EPS).map(logit)
    vs["logit_p_vid_30d"] = vs["p_vid_30d"].clip(EPS, 1-EPS).map(logit)

    vs["dlogit_p_1d_7d"]  = vs["logit_p_vid_1d"]  - vs["logit_p_vid_7d"]
    vs["dlogit_p_7d_30d"] = vs["logit_p_vid_7d"]  - vs["logit_p_vid_30d"]

    # тренды просмотров
    vs["trend_views_d_w"] = np.log1p(vs["views_1d"]) - np.log1p(vs["views_7d"])
    vs["trend_p_1d_7d"]   = np.log(vs["p_vid_1d"].clip(EPS,1)) - np.log(vs["p_vid_7d"].clip(EPS,1))

    # ---------- 2) Автор/категория агрегаты для backoff ----------
    # суммарные lv/views по авторам и категориям
    agg_author = vs.groupby("author_id", dropna=False).agg(
        lv_1d=("lv_1d","sum"), views_1d=("views_1d","sum"),
        lv_7d=("lv_7d","sum"), views_7d=("views_7d","sum"),
        lv_30d=("lv_30d","sum"), views_30d=("views_30d","sum"),
    ).reset_index()
    agg_cat = vs.groupby("category_id", dropna=False).agg(
        lv_1d=("lv_1d","sum"), views_1d=("views_1d","sum"),
        lv_7d=("lv_7d","sum"), views_7d=("views_7d","sum"),
        lv_30d=("lv_30d","sum"), views_30d=("views_30d","sum"),
    ).reset_index()

    for w in ("1d","7d","30d"):
        agg_author[f"p_author_{w}"] = beta_smooth(agg_author[f"lv_{w}"], agg_author[f"views_{w}"], cfg.alpha_author, p0_global)
        agg_cat[f"p_category_{w}"]  = beta_smooth(agg_cat[f"lv_{w}"],    agg_cat[f"views_{w}"],   cfg.alpha_category, p0_global)

    vs = vs.merge(agg_author[["author_id","p_author_1d","p_author_7d","p_author_30d","views_1d","views_7d","views_30d"]]
                  .rename(columns={"views_1d":"author_views_1d","views_7d":"author_views_7d","views_30d":"author_views_30d"}),
                  on="author_id", how="left")
    vs = vs.merge(agg_cat[["category_id","p_category_1d","p_category_7d","p_category_30d","views_1d","views_7d","views_30d"]]
                  .rename(columns={"views_1d":"cat_views_1d","views_7d":"cat_views_7d","views_30d":"cat_views_30d"}),
                  on="category_id", how="left")

    # residuals (сложность видео относительно автора/категории)
    vs["res_vid_vs_author_7d"] = vs["logit_p_vid_7d"] - vs["p_author_7d"].clip(EPS,1-EPS).map(logit)
    vs["res_vid_vs_cat_7d"]    = vs["logit_p_vid_7d"] - vs["p_category_7d"].clip(EPS,1-EPS).map(logit)

    # engagement (логиты CR по окнам)
    def cr_logit(cols: Iterable[str], df: pd.DataFrame) -> pd.Series:
        vals = []
        for c in cols:
            if c in df.columns:
                x = pct01(df[c].astype(float))
                x = x.clip(EPS, 1-EPS)
                vals.append(x.map(logit))
        if not vals:
            return pd.Series(np.nan, index=df.index)
        return pd.concat(vals, axis=1).mean(axis=1)

    vs["engagement_1d"]  = cr_logit(["v_cr_click_like_1_days","v_cr_click_comment_1_days","v_cr_click_vtop_1_days"], vs)
    vs["engagement_7d"]  = cr_logit(["v_cr_click_like_7_days","v_cr_click_comment_7_days","v_cr_click_vtop_7_days"], vs)
    vs["engagement_30d"] = cr_logit(["v_cr_click_like_30_days","v_cr_click_comment_30_days","v_cr_click_vtop_30_days"], vs)

    # ---------- 3) Optional: эмбеддинги, автор/категорияные центроиды ----------
    emb_map: Dict = {}
    author_centroids = {}
    category_centroids = {}
    if video_emb is not None:
        em = video_emb.copy()
        # ожидаем колонки: video_id, emb (np.ndarray)
        em["emb"] = em["emb"].map(ensure_array)
        emb_map = dict(zip(em["video_id"].values, em["emb"].values))

        # центроиды "позитив/негатив" по авторам/категориям через p_vid_7d как вес
        # (быстрая приближенка без train-лейблов)
        def weighted_centroids(key_col: str):
            # объединяем vs с эмбеддингами
            tmp = vs.merge(em[["video_id","emb"]], on="video_id", how="inner")
            # веса позитивов/негативов
            w_pos = tmp["p_vid_7d"].values
            w_neg = (1.0 - tmp["p_vid_7d"].values)
            # аккум в dict: key -> (sum_pos, wpos_sum, sum_neg, wneg_sum)
            acc = {}
            for k, e, wp, wn in zip(tmp[key_col].values, tmp["emb"].values, w_pos, w_neg):
                if e is None:
                    continue
                if k not in acc:
                    acc[k] = [np.zeros_like(e, dtype="float32"), 0.0,
                              np.zeros_like(e, dtype="float32"), 0.0]
                acc[k][0] += e * wp; acc[k][1] += wp
                acc[k][2] += e * wn; acc[k][3] += wn
            out = {}
            for k,(sp,wp,sn,wn) in acc.items():
                cp = sp / (wp + 1e-9) if wp > 0 else None
                cn = sn / (wn + 1e-9) if wn > 0 else None
                out[k] = (cp, cn)
            return out

        author_centroids   = weighted_centroids("author_id")
        category_centroids = weighted_centroids("category_id")

    # ---------- 4) Собираем общий поток (train+test) для as-of онлайна ----------
    train["is_train"] = 1
    test["is_train"]  = 0
    all_df = pd.concat([train, test], ignore_index=True, sort=False)

    # Примержим статические видео-фичи
    cols_keep_vs = [
        "video_id","author_id","category_id","v_duration","v_pub_datetime",
        "views_1d","views_7d","views_30d",
        "p_vid_1d","p_vid_7d","p_vid_30d",
        "logit_p_vid_1d","logit_p_vid_7d","logit_p_vid_30d",
        "dlogit_p_1d_7d","dlogit_p_7d_30d",
        "p_author_1d","p_author_7d","p_author_30d",
        "p_category_1d","p_category_7d","p_category_30d",
        "author_views_1d","author_views_7d","author_views_30d",
        "cat_views_1d","cat_views_7d","cat_views_30d",
        "v_avg_watchtime_1_day","v_avg_watchtime_7_day","v_avg_watchtime_30_day",
        "v_frac_avg_watchtime_1_day_duration","v_frac_avg_watchtime_7_day_duration","v_frac_avg_watchtime_30_day_duration",
        "engagement_1d","engagement_7d","engagement_30d",
        "trend_views_d_w","trend_p_1d_7d",
        "res_vid_vs_author_7d","res_vid_vs_cat_7d",
    ]
    all_df = all_df.merge(vs[cols_keep_vs], on="video_id", how="left")

    # Векторные статфичи от события
    all_df["time_of_day"] = all_df["event_dt"].dt.hour.astype("int16")
    if cfg.make_time_bins:
        # грубые бины: ночь(0-5), утро(6-11), день(12-17), вечер(18-23)
        tod = all_df["time_of_day"].values
        all_df["time_bin"] = pd.cut(tod, bins=[-1,5,11,17,24], labels=[0,1,2,3]).astype("int8")
    else:
        all_df["time_bin"] = -1

    # возраст видео на момент события
    all_df["age_hours"] = (all_df["event_dt"] - all_df["v_pub_datetime"]).dt.total_seconds() / 3600.0

    # тип порога (<=5 мин)
    all_df["threshold_type"] = (all_df["v_duration"] <= 300).astype("int8")

    # backoff выбор p_*: видео → автор → категория → глобаль
    def pick_backoff(row, w: str):
        # row содержит: views_{w}, p_vid_{w}, author_views_{w}, p_author_{w}, cat_views_{w}, p_category_{w}
        vviews = row[f"views_{w}"]
        if pd.notna(vviews) and vviews >= getattr(cfg, f"min_views_video"):
            return row[f"p_vid_{w}"]
        aviews = row[f"author_views_{w}"]
        if pd.notna(aviews) and aviews >= getattr(cfg, f"min_views_author"):
            return row[f"p_author_{w}"]
        cviews = row[f"cat_views_{w}"]
        if pd.notna(cviews) and cviews >= getattr(cfg, f"min_views_category"):
            return row[f"p_category_{w}"]
        return p0_global

    for w in ("1d","7d","30d"):
        all_df[f"p_backoff_{w}"] = all_df.apply(lambda r: pick_backoff(r, w), axis=1)
        all_df[f"logit_p_backoff_{w}"] = all_df[f"p_backoff_{w}"].clip(EPS,1-EPS).map(logit)

    # взаимодействия
    all_df["dur_x_p1d"]   = all_df["v_duration"] * all_df["p_backoff_1d"]
    all_df["thr_x_p1d"]   = all_df["threshold_type"] * all_df["p_backoff_1d"]
    all_df["dur_x_ease7"] = all_df["v_duration"] * all_df["v_frac_avg_watchtime_7_day_duration"].fillna(0)

    # ---------- 5) Online (as-of) фичи по пользователю, региону и т.п. ----------
    # Состояния
    last_time = {}                     # user -> last event ts
    last_gap_deque = defaultdict(lambda: deque(maxlen=10))  # user -> deque последних Δt
    session_len = defaultdict(int)     # user -> len текущей сессии
    seen_pairs = set()                 # {(user,video)} seen
    prev_watch_map = {}                # (user,video) -> last watchtime

    # пользовательские исходы
    user_outcomes = defaultdict(lambda: deque(maxlen=max(cfg.user_L_list)))  # user -> deque[0/1]
    user_times = defaultdict(lambda: deque(maxlen=10))  # для медианы скоростей

    # user×author/category счётчики
    ua_pos = defaultdict(int); ua_tot = defaultdict(int)
    uc_pos = defaultdict(int); uc_tot = defaultdict(int)

    # гео-агрегаты
    region_pos = defaultdict(int); region_tot = defaultdict(int)
    city_pos = defaultdict(int);   city_tot = defaultdict(int)
    region_hour_pos = defaultdict(int); region_hour_tot = defaultdict(int)

    # свежесть
    user_fresh_pos = defaultdict(int); user_fresh_tot = defaultdict(int)

    # эмбеддинги — пользовательские центроиды и недавние
    user_pos_c = {}   # user -> np.ndarray
    user_neg_c = {}   # user -> np.ndarray
    user_recent_emb = defaultdict(lambda: deque(maxlen=cfg.recent_k_sim))

    # Подготовка списков, чтобы заполнять без .loc в цикле
    n = len(all_df)
    out = {
        # user history rates
        **{f"user_long_rate_L{L}": [np.nan]*n for L in cfg.user_L_list},
        "user_recent_gap_s": [np.nan]*n,
        "session_len": [0]*n,
        "user_speed_median_gap_s": [np.nan]*n,

        # seen/repeat
        "seen_same_video_before": [0]*n,
        "prev_watchtime_same_video": [np.nan]*n,

        # affinity user×author/category
        "user_author_long_rate": [np.nan]*n,
        "user_category_long_rate": [np.nan]*n,

        # geo rates
        "region_long_rate": [np.nan]*n,
        "city_long_rate": [np.nan]*n,
        "region_hour_long_rate": [np.nan]*n,

        # fresh pref
        "user_fresh_long_rate": [np.nan]*n,

        # embedding sims (optional)
        "sim_user_pos": [np.nan]*n,
        "sim_user_neg": [np.nan]*n,
        "sim_user_margin": [np.nan]*n,
        "sim_user_recent_k": [np.nan]*n,
        "sim_author_pos_margin": [np.nan]*n,
        "sim_category_pos_margin": [np.nan]*n,

        # interactions online
        "dur_x_user_speed": [np.nan]*n,
    }

    # prefetch columns as arrays (ускоряет)
    arr_user   = all_df["user_id"].values
    arr_video  = all_df["video_id"].values
    arr_region = all_df["region"].astype(str).fillna("Unknown").values
    arr_city   = all_df["city"].astype(str).fillna("Unknown").values
    arr_time   = all_df["event_dt"].astype("datetime64[s]").astype(np.int64)  # ns->s int
    arr_hour   = all_df["time_of_day"].values
    arr_is_tr  = all_df["is_train"].values
    arr_watch  = all_df["watchtime"].values if "watchtime" in all_df.columns else np.array([np.nan]*n)
    arr_dur    = all_df["v_duration"].values
    arr_age_h  = all_df["age_hours"].values
    arr_author = all_df["author_id"].values
    arr_cat    = all_df["category_id"].values

    # emb helpers (author/category centroids)
    def author_pos_neg_centroids(aid):
        if not author_centroids: return (None, None)
        return author_centroids.get(aid, (None, None))
    def category_pos_neg_centroids(cid):
        if not category_centroids: return (None, None)
        return category_centroids.get(cid, (None, None))

    # главное: цикл по событиям по времени
    # сортируем индексы по времени (стабильно)
    order = np.argsort(arr_time, kind="stable")
    for j, idx in enumerate(order):
        u = arr_user[idx]
        v = arr_video[idx]
        r = arr_region[idx]
        c = arr_city[idx]
        t = int(arr_time[idx])
        hr = int(arr_hour[idx])
        is_tr = int(arr_is_tr[idx])
        wt = arr_watch[idx] if is_tr == 1 else np.nan
        dur = float(arr_dur[idx]) if not pd.isna(arr_dur[idx]) else np.nan
        ageh = float(arr_age_h[idx]) if not pd.isna(arr_age_h[idx]) else np.nan
        aid = arr_author[idx]
        cid = arr_cat[idx]

        # ---- seen / prev watch
        seen = 1 if (u, v) in seen_pairs else 0
        out["seen_same_video_before"][idx] = seen
        out["prev_watchtime_same_video"][idx] = prev_watch_map.get((u, v), np.nan)

        # ---- gaps/speeds/session
        last_t = last_time.get(u, None)
        gap = (t - last_t) if last_t is not None else np.nan
        out["user_recent_gap_s"][idx] = gap

        if last_t is None or (gap is not None and gap > cfg.session_gap_s):
            session_len[u] = 1
            last_gap_deque[u].clear()
            user_times[u].clear()
        else:
            session_len[u] = session_len[u] + 1

        out["session_len"][idx] = session_len[u]
        if not np.isnan(gap):
            last_gap_deque[u].append(gap)
            user_times[u].append(t)

        # медиана последних Δt
        if len(last_gap_deque[u]) > 0:
            out["user_speed_median_gap_s"][idx] = float(np.median(last_gap_deque[u]))

        # ---- user L-rates
        outcomes = user_outcomes[u]
        for L in cfg.user_L_list:
            if len(outcomes) == 0:
                out[f"user_long_rate_L{L}"][idx] = np.nan
            else:
                # берем последние L (или все, если меньше)
                arr = list(outcomes)[-L:]
                out[f"user_long_rate_L{L}"][idx] = float(sum(arr)/len(arr))

        # ---- user×author/category smoothed rates
        key_ua = (u, aid)
        key_uc = (u, cid)
        p_ua = beta_smooth(ua_pos[key_ua], ua_tot[key_ua], cfg.alpha_online_small, p0_global)
        p_uc = beta_smooth(uc_pos[key_uc], uc_tot[key_uc], cfg.alpha_online_small, p0_global)
        out["user_author_long_rate"][idx] = p_ua
        out["user_category_long_rate"][idx] = p_uc

        # ---- region/city/region×hour rates
        p_reg = beta_smooth(region_pos[r], region_tot[r], cfg.alpha_online_small, p0_global)
        p_city = beta_smooth(city_pos[c], city_tot[c], cfg.alpha_online_small, p0_global)
        p_reg_hr = beta_smooth(region_hour_pos[(r,hr)], region_hour_tot[(r,hr)], cfg.alpha_online_small, p0_global)
        out["region_long_rate"][idx] = p_reg
        out["city_long_rate"][idx] = p_city
        out["region_hour_long_rate"][idx] = p_reg_hr

        # ---- fresh pref (<= fresh_hours)
        p_fresh = beta_smooth(user_fresh_pos[u], user_fresh_tot[u], cfg.alpha_online_small, p0_global)
        out["user_fresh_long_rate"][idx] = p_fresh

        # ---- embedding sims (optional)
        e = emb_map.get(v, None) if emb_map else None
        # пользовательские pos/neg центроиды
        up = user_pos_c.get(u, None)
        un = user_neg_c.get(u, None)
        if e is not None:
            out["sim_user_pos"][idx] = cos_sim(e, up) if up is not None else np.nan
            out["sim_user_neg"][idx] = cos_sim(e, un) if un is not None else np.nan
            if (up is not None) or (un is not None):
                out["sim_user_margin"][idx] = (cos_sim(e, up) if up is not None else 0.0) - (cos_sim(e, un) if un is not None else 0.0)
            # недавние k
            if len(user_recent_emb[u]) > 0:
                mean_recent = np.mean(np.stack(list(user_recent_emb[u]), axis=0), axis=0)
                out["sim_user_recent_k"][idx] = cos_sim(e, mean_recent)
            # автор/категория марджины
            ap, an = author_pos_neg_centroids(aid)
            cp, cn = category_pos_neg_centroids(cid)
            if ap is not None or an is not None:
                out["sim_author_pos_margin"][idx] = (cos_sim(e, ap) if ap is not None else 0.0) - (cos_sim(e, an) if an is not None else 0.0)
            if cp is not None or cn is not None:
                out["sim_category_pos_margin"][idx] = (cos_sim(e, cp) if cp is not None else 0.0) - (cos_sim(e, cn) if cn is not None else 0.0)

        # ---- interactions online
        sp = out["user_speed_median_gap_s"][idx]
        out["dur_x_user_speed"][idx] = (dur * sp) if (not pd.isna(dur) and not pd.isna(sp)) else np.nan

        # ==================== UPDATE STATE (после вычисления фичей) ====================
        # seen/prev watch — не зависит от лейбла
        seen_pairs.add((u, v))
        if is_tr == 1 and not pd.isna(wt):
            prev_watch_map[(u, v)] = float(wt)

        # обновляем время/сессию
        last_time[u] = t

        # эмбеддинги: добавим в "recent"
        if e is not None:
            user_recent_emb[u].append(e)

        # лейбл для train
        if is_tr == 1 and not pd.isna(wt) and not pd.isna(dur):
            if dur > 300:
                y = 1 if (wt > 0.25 * dur) else 0
            else:
                y = 1 if (wt > 30.0) else 0

            # user outcomes
            user_outcomes[u].append(y)

            # user×author/category
            ua_tot[key_ua] += 1; uc_tot[key_uc] += 1
            if y == 1:
                ua_pos[key_ua] += 1; uc_pos[key_uc] += 1

            # geo
            region_tot[r] += 1; city_tot[c] += 1; region_hour_tot[(r,hr)] += 1
            if y == 1:
                region_pos[r] += 1; city_pos[c] += 1; region_hour_pos[(r,hr)] += 1

            # fresh pref
            if not pd.isna(ageh) and ageh <= cfg.fresh_hours:
                user_fresh_tot[u] += 1
                if y == 1:
                    user_fresh_pos[u] += 1

            # эмбеддинги: EMA по pos/neg центроидам
            if e is not None:
                if y == 1:
                    user_pos_c[u] = ema_update(user_pos_c.get(u, None), e, cfg.ema_beta)
                else:
                    user_neg_c[u] = ema_update(user_neg_c.get(u, None), e, cfg.ema_beta)

    # приклеим онлайновые фичи
    for k, vals in out.items():
        all_df[k] = vals

    # ---------- 6) Target для train, финальный отбор колонок ----------
    # целевая переменная по правилам
    if "watchtime" in train.columns:
        msk_tr = all_df["is_train"] == 1
        dur = all_df.loc[msk_tr, "v_duration"].values
        wt  = all_df.loc[msk_tr, "watchtime"].values
        y = np.zeros(msk_tr.sum(), dtype=np.int8)
        longer = dur > 300
        y[longer]  = (wt[longer] > 0.25 * dur[longer]).astype(np.int8)
        y[~longer] = (wt[~longer] > 30.0).astype(np.int8)
        all_df.loc[msk_tr, "target"] = y

    # финальный список фичей
    feat_cols = [
        # видео-базлайн
        "p_backoff_1d","p_backoff_7d","p_backoff_30d",
        "logit_p_backoff_1d","logit_p_backoff_7d","logit_p_backoff_30d",
        "p_vid_1d","p_vid_7d","p_vid_30d",
        "dlogit_p_1d_7d","dlogit_p_7d_30d",
        "v_frac_avg_watchtime_1_day_duration","v_frac_avg_watchtime_7_day_duration","v_frac_avg_watchtime_30_day_duration",
        "engagement_1d","engagement_7d","engagement_30d",
        "trend_views_d_w","trend_p_1d_7d",
        "res_vid_vs_author_7d","res_vid_vs_cat_7d",

        # статические от события
        "v_duration","age_hours","time_of_day","time_bin","threshold_type",
        "dur_x_p1d","thr_x_p1d","dur_x_ease7",

        # онлайн-пользовательские
        *[f"user_long_rate_L{L}" for L in cfg.user_L_list],
        "user_recent_gap_s","session_len","user_speed_median_gap_s",
        "seen_same_video_before","prev_watchtime_same_video",
        "user_author_long_rate","user_category_long_rate",
        "region_long_rate","city_long_rate","region_hour_long_rate",
        "user_fresh_long_rate",
        "dur_x_user_speed",

        # эмбеддинги (если были)
        "sim_user_pos","sim_user_neg","sim_user_margin","sim_user_recent_k",
        "sim_author_pos_margin","sim_category_pos_margin",
    ]

    # часть колонок может отсутствовать (если не было эмбеддингов) — отфильтруем
    feat_cols = [c for c in feat_cols if c in all_df.columns]

    train_feat = all_df[all_df["is_train"] == 1].copy()
    test_feat  = all_df[all_df["is_train"] == 0].copy()

    # safety: fillna
    train_feat[feat_cols] = train_feat[feat_cols].replace([np.inf,-np.inf], np.nan).fillna(0)
    test_feat[feat_cols]  = test_feat[feat_cols].replace([np.inf,-np.inf], np.nan).fillna(0)

    # вернём только нужные
    keep_train = ["user_id","video_id","event_timestamp","target"] + feat_cols
    keep_test  = ["user_id","video_id","event_timestamp"] + feat_cols
    train_feat = train_feat[keep_train]
    test_feat  = test_feat[keep_test]

    return train_feat, test_feat


In [26]:
# video_emb: DataFrame со столбцами ['video_id','emb'], где emb — np.ndarray одинаковой длины
video_emb = None  # или свой мэппинг эмбеддингов


In [ ]:
import re
import numpy as np
import pandas as pd

def add_text_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # гарантируем текстовые поля
    for col in ["title", "description"]:
        if col not in df.columns:
            df[col] = ""
        df[col] = df[col].fillna("").astype(str)
        df[f"{col}_lc"] = df[col].str.lower()

    # если нет длительности — заполним NaN (используется только для нормировок)
    if "v_duration" not in df.columns:
        df["v_duration"] = np.nan
    df["v_duration"] = df["v_duration"].astype(float)

    # длины
    df["title_len_char"]  = df["title"].str.len().fillna(0).astype("int32")
    df["desc_len_char"]   = df["description"].str.len().fillna(0).astype("int32")
    df["title_len_words"] = df["title"].str.split().str.len().fillna(0).astype("int32")
    df["desc_len_words"]  = df["description"].str.split().str.len().fillna(0).astype("int32")
    df["title_avg_word_len"] = (df["title_len_char"] / (df["title_len_words"] + 1e-6)).astype("float32")
    df["desc_avg_word_len"]  = (df["desc_len_char"]  / (df["desc_len_words"]  + 1e-6)).astype("float32")

    # пунктуация / цифры
    df["title_exclam"]   = df["title"].str.count("!").fillna(0).astype("int16")
    df["title_question"] = df["title"].str.count(r"\?").fillna(0).astype("int16")
    df["desc_exclam"]    = df["description"].str.count("!").fillna(0).astype("int16")
    df["desc_question"]  = df["description"].str.count(r"\?").fillna(0).astype("int16")
    df["title_digits"]   = df["title"].str.count(r"\d").fillna(0).astype("int16")
    df["desc_digits"]    = df["description"].str.count(r"\d").fillna(0).astype("int16")

    # URL / таймкоды / главы
    url_pat   = r"(https?://|www\.)"
    tcode_pat = r"\b(?:(?:\d{1,2}:){1,2}\d{2})\b"  # mm:ss или hh:mm:ss
    df["title_has_url"]      = df["title_lc"].str.contains(url_pat, regex=True).fillna(False).astype("int8")
    df["desc_has_url"]       = df["description_lc"].str.contains(url_pat, regex=True).fillna(False).astype("int8")
    df["title_has_timecode"] = df["title_lc"].str.contains(tcode_pat, regex=True).fillna(False).astype("int8")
    df["desc_timecode_count"]= df["description_lc"].str.count(tcode_pat).fillna(0).astype("int16")
    df["desc_has_timecode"]  = (df["desc_timecode_count"] > 0).astype("int8")
    df["desc_has_chapters"]  = (df["desc_timecode_count"] >= 3).astype("int8")  # 3+ таймкодов ~ главы

    # ключевые слова
    KW = {
        "live":      ["live", "стрим", "прямая трансляция", "трансляция"],
        "trailer":   ["trailer", "трейлер"],
        "podcast":   ["podcast", "подкаст"],
        "interview": ["интервью", "interview"],
        "lesson":    ["how to", "tutorial", "урок", "гайд", "обучение", "инструкция", "guide"],
        "full":      ["полная версия", "full version", "full movie", "без сокращений", "без рекламы"],
        "series":    ["серия", "выпуск", "эпизод", "season", "episode", "ep"],
        "music":     ["official video", "official audio", "lyrics", "клип", "remix"],
    }
    def any_kw(series_lc: pd.Series, kws) -> pd.Series:
        pat = "|".join(re.escape(k) for k in kws)
        return series_lc.str.contains(pat, regex=True)

    for k, lst in KW.items():
        df[f"title_kw_{k}"] = any_kw(df["title_lc"], lst).fillna(False).astype("int8")
        df[f"desc_kw_{k}"]  = any_kw(df["description_lc"], lst).fillna(False).astype("int8")

    # агрегаты по ключевым словам
    info_keys = ["podcast", "interview", "lesson", "full", "series"]
    df["kw_info_longform"] = df[[f"title_kw_{k}" for k in info_keys] + [f"desc_kw_{k}" for k in info_keys]].max(axis=1).astype("int8")
    df["kw_live_any"]      = df[["title_kw_live", "desc_kw_live"]].max(axis=1).astype("int8")
    df["kw_trailer_any"]   = df[["title_kw_trailer", "desc_kw_trailer"]].max(axis=1).astype("int8")
    df["kw_music_any"]     = df[["title_kw_music", "desc_kw_music"]].max(axis=1).astype("int8")

    # нормировки на длительность
    minutes = (df["v_duration"] / 60.0).astype("float32")
    df["title_words_per_min"] = (df["title_len_words"] / (minutes + 1e-6)).astype("float32")
    df["desc_words_per_min"]  = (df["desc_len_words"]  / (minutes + 1e-6)).astype("float32")

    # взаимодействие с ease7 (если колонка есть; иначе 0)
    ease7 = df["v_frac_avg_watchtime_7_day_duration"].astype(float) if "v_frac_avg_watchtime_7_day_duration" in df.columns else 0.0
    df["ease7_x_desc_len"] = (df["desc_len_words"] * ease7).astype("float32")

    return df

# === Применяем ТОЛЬКО текстовые фичи к вашим датафреймам ===
train_full_new1 = add_text_features(train_full_new1)
test_full_new1  = add_text_features(test_full_new1)


# SKF catboost

In [ ]:
useless_cols=['event_timestamp',
          'v_pub_datetime', ]

In [ ]:
train_full_new1=train_full_new1.drop(columns=useless_cols)

test_full_new1=test_full_new1.drop(columns=useless_cols)

In [ ]:
train_full_new1=train_full_new1[:3_000_000]

In [ ]:
X=train_full_new1.drop(columns=['watchtime', 'y_long', 'longwatch_threshold_sec', 'title', 'description'])
y=train_full_new1['y_long']


In [ ]:
X_train, X_eval, y_train, y_eval=train_test_split(X, y, test_size=0.15, random_state=seed)

In [ ]:
#for now just one catboost 

model=CatBoostClassifier(
    iterations=1000,
    depth=6,
    learning_rate=0.03,
    loss_function='Logloss',
    eval_metric='F1',
    task_type='GPU',
    #l2_leaf_reg=,
    
)

In [ ]:
cat_feats=[ 'user_id','video_id','category_id','author_id',  'region','city']

In [ ]:
model.fit(X_train, y_train, eval_set=(X_eval, y_eval),cat_features=cat_feats, verbose=200, early_stopping_rounds=500)

# Just mean of them 

In [ ]:
num_feats=train_full_new1.select_dtypes(include='number').columns.tolist()


In [ ]:
corr=train_full_new1[num_feats].corrwith(train_full_new1['y_long'])

In [ ]:
corr

In [ ]:
X_test=test_full_new1.drop(columns=['title', 'description'])

In [ ]:
preds=model.predict(X_test)

In [ ]:
sample['target']=preds

In [ ]:
sample.to_csv('submission.csv', index=False)

# Meta Models